In [1]:
import torch

In [2]:
import numpy as np

In [5]:
x = torch.empty(5, 3)
x

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.4013e-45,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.9740e+36, -6.4572e-42],
        [ 8.8910e-10,  0.0000e+00,  0.0000e+00]])

In [7]:
x = torch.rand(5, 3)
x

tensor([[0.4226, 0.7826, 0.0062],
        [0.7175, 0.2731, 0.5159],
        [0.1288, 0.1873, 0.0846],
        [0.4868, 0.7517, 0.7624],
        [0.5561, 0.3623, 0.7808]])

In [9]:
x = x.new_ones(2, 3)
x

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [17]:
x = torch.randn_like(x)
x

tensor([[-1.7917],
        [-0.4879],
        [-1.2795],
        [-1.2009],
        [-0.1148],
        [-0.6892]])

In [18]:
x = x.view(1, 6)
x

tensor([[-1.7917, -0.4879, -1.2795, -1.2009, -0.1148, -0.6892]])

In [19]:
x = x.view(6, 1)
x

tensor([[-1.7917],
        [-0.4879],
        [-1.2795],
        [-1.2009],
        [-0.1148],
        [-0.6892]])

In [20]:
x[0].item()

-1.7917137145996094

In [22]:
a = x.numpy()
a

array([[-1.7917137 ],
       [-0.48787943],
       [-1.2795258 ],
       [-1.2008926 ],
       [-0.11483982],
       [-0.6892369 ]], dtype=float32)

In [27]:
x = x + 1
x

tensor([[0.2083],
        [1.5121],
        [0.7205],
        [0.7991],
        [1.8852],
        [1.3108]])

In [28]:
a

array([[-1.7917137 ],
       [-0.48787943],
       [-1.2795258 ],
       [-1.2008926 ],
       [-0.11483982],
       [-0.6892369 ]], dtype=float32)

In [30]:
b = np.ones(5)

In [31]:
torch.from_numpy(b)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

In [40]:
z = x + x.new_ones(6, 1)
z

tensor([[1.2083],
        [2.5121],
        [1.7205],
        [1.7991],
        [2.8852],
        [2.3108]])

In [44]:
b = torch.ones((5,2), requires_grad=True)
b

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], requires_grad=True)

In [46]:
c = b + 2
c

tensor([[3., 3.],
        [3., 3.],
        [3., 3.],
        [3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)

In [48]:
d = b * c
d

tensor([[3., 3.],
        [3., 3.],
        [3., 3.],
        [3., 3.],
        [3., 3.]], grad_fn=<ThMulBackward>)

In [49]:
e = np.array(range(4))
e

array([0, 1, 2, 3])

In [52]:
e.unsqueeze()

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [54]:
et = torch.from_numpy(e)
et

tensor([0, 1, 2, 3], dtype=torch.int32)

In [65]:
et.unsqueeze(0)

tensor([[0, 1, 2, 3]], dtype=torch.int32)

In [66]:
e.data


In [63]:
torch.randn(1, 3)

tensor([[0.4956, 0.2413, 0.3092]])

In [65]:
#从正太分布中随机取
torch.randn?

In [66]:
#从uniform分布中随机取
torch.rand?

### CBOW model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [55]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_dim)
        self.project = nn.Linear(embed_dim, vocab_size)
    
    def forward(self, inputs):
        embeds = torch.sum((self.embeddings(inputs)), 0, True)
        #print(embeds.size())
        out = F.relu(self.project(embeds))
        log_prob = F.log_softmax(out, dim=1)
        return log_prob

In [60]:
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [61]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


#make_context_vector(data[0][0], word_to_ix)  # example

In [62]:
for epoch in range(10):
    total_loss = 0
    for context, target in data:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = make_context_vector(context, word_to_ix)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        #print(model)
        #print(context_idxs)
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[240.64217257499695, 234.51366567611694, 228.6160650253296, 222.94700622558594, 217.50964260101318, 212.30550527572632, 207.1886591911316, 202.04871487617493, 197.1684398651123, 192.55007123947144]
